
In this exercise, your goal is to implement a `Dataset` class in PyTorch, which is used to handle and process the data for training a transformer model. You'll also use a `DataLoader` to load batches of data from this dataset. Follow the steps below to complete the implementation based on the provided starter code.

The starter code includes the tokenizer that we've implemented in previous videos. You will need to create an instance of this tokenizer using the training dataset.

To implement the exercise go over the code and implement the missing parts marked with the `TODO` comment.

## 1. **Define the `TokenIdsDataset` Class:**

For the first step, you would need to implement the missing methods in the `TokenIdsDataset` class.

- **`__init__` Method:**
    - Initialize the class with `data` and `block_size`.
    - Save `data` and `block_size` as instance variables.

- **`__len__` Method:**
    - Compute the size of the dataset. If every position in the data can be the start of an item, the length of the dataset should be less than the `len(data)`.

- **`__getitem__` Method:**
    - Validate the input position to ensure it is within a valid range.
    - Retrieve an item starting from position `pos` up to `pos + block_size`.
    - Retrieve a target item that is the same as the item but has been shifted by one position.
    - Return both the input item and the target item.

## 2. **Tokenize the Text:**

Create a tokenizer and encode data from the training dataset. Then, create an instance of the `TokenIdsDataset` for the training data.

## 3. **Retrieve the First Item from the Dataset**

Get the first item from the dataset and decode it using the tokenizer. If everything is implemented correctly, you should get the first 64 characters of the training dataset.

## 4. **Use a DataLoader:**

Now, try using the `DataLoader` with the training dataset we've created. The `DataLoader` here is created with a `RandomSampler` to randomize the items we get from the training dataset.

For this exercise, first, get a single training batch from the `DataLoader` we've created.

Then, we decode input and target tokens using the tokenizer we've created. The input and target should be from the same part of the training dataset but shifted by one character.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Use CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from pathlib import Path

text = Path('tiny-shakespeare.txt').read_text()

In [ ]:
print(text[0:1000])

In [ ]:

class CharTokenizer:
  def __init__(self, vocabulary):
    self.token_id_for_char = {char: token_id for token_id, char in enumerate(vocabulary)}
    self.char_for_token_id = {token_id: char for token_id, char in enumerate(vocabulary)}

  @staticmethod
  def train_from_text(text):
    vocabulary = set(text)
    return CharTokenizer(sorted(list(vocabulary)))

  def encode(self, text):
    token_ids = []
    for char in text:
      token_ids.append(self.token_id_for_char[char])
    return torch.tensor(token_ids, dtype=torch.long)

  def decode(self, token_ids):
    chars = []
    for token_id in token_ids.tolist():
      chars.append(self.char_for_token_id[token_id])
    return ''.join(chars)


  def vocabulary_size(self):
    return len(self.token_id_for_char)

In [ ]:
tokenizer = CharTokenizer.train_from_text(text)

In [ ]:
print(tokenizer.encode("Hello world"))
print(tokenizer.decode(tokenizer.encode("Hello world")))

In [ ]:
print(f"Vocabulary size: {tokenizer.vocabulary_size()}")

In [ ]:
from torch.utils.data import Dataset

class TokenIdsDataset(Dataset):
  def __init__(self, data, block_size):
    self.data = data
    self.block_size = block_size

  def __len__(self):
    return len(self.data) - self.block_size

  def __getitem__(self, pos):
    assert pos < len(self.data) - self.block_size

    x = self.data[pos:pos + self.block_size]
    y = self.data[pos + 1:pos + 1 + self.block_size]
    return x, y

In [ ]:
tokenized_text = tokenizer.encode(text)
dataset = TokenIdsDataset(tokenized_text, block_size=64)

In [ ]:
x, y = dataset[0]

In [ ]:
x

In [ ]:
tokenizer.decode(x)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler

sampler = RandomSampler(dataset, replacement=True)
dataloader = DataLoader(dataset, batch_size=2, sampler=sampler)

In [ ]:
x, y = next(iter(dataloader))

In [ ]:
x.shape

In [ ]:
x

In [ ]:
tokenizer.decode(x[0])

In [ ]:
tokenizer.decode(y[0])